In [1]:
import pandas as pd
import numpy as np
import win32com.client
import re
import datetime
import os
from dotenv import load_dotenv
load_dotenv()
import sqlalchemy
import mysql.connector

1. ####  Create file with list of required packages

In [2]:
with open('requirements.txt', 'w') as f:
    f.write('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))

2. #### Downlaod email attachment

In [3]:
def read_email():
    outlook_mail_client = win32com.client.Dispatch('Outlook.Application').GetNamespace('MAPI')
    inbox = outlook_mail_client.GetDefaultFolder(6)
    messages = inbox.Items
    path = 'E:/ACTIVE/Python/experian_data-service'
    message = messages.GetFirst()
    today_date = str(datetime.date.today())
    while True:
        try:
            email_subject = 'SUBJECT-TITLE'
            subject = str(message.Subject).lower()

           # Search email with subject

            if re.search(email_subject, subject) != None:
                attachments = message.Attachments
                attachment = str(attachments.Item(1)).lower()
                attachment.SaveASFile(path + '\\' + attachment_name)
            else:
                pass
            message = messages.GetNext()
        except:
            message = messages.GetNext()
    exit

In [4]:
#read_email()

4. #### Read IBRD dataset

In [5]:
def read_data(filename):
    if filename.endswith('csv'):
        df = pd.read_csv(filename)
    return df

In [6]:
filepath = os.getenv('file_path')
filename = os.environ.get('file_name')
full_file_name = filepath+filename
data =  read_data(full_file_name)

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8986 entries, 0 to 8985
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   End of Period                 8986 non-null   object 
 1   Loan Number                   8986 non-null   object 
 2   Region                        8986 non-null   object 
 3   Country Code                  8984 non-null   object 
 4   Country                       8986 non-null   object 
 5   Borrower                      8929 non-null   object 
 6   Guarantor Country Code        8704 non-null   object 
 7   Guarantor                     8706 non-null   object 
 8   Loan Type                     8986 non-null   object 
 9   Loan Status                   8986 non-null   object 
 10  Interest Rate                 8865 non-null   float64
 11  Currency of Commitment        0 non-null      float64
 12  Project ID                    8660 non-null   object 
 13  Pro

In [8]:
# 303 loans with zero principal
zero_principal = data[data["Original Principal Amount"] == 0].shape[0]
zero_principal


303

In [9]:
# 2437 loans with zero interest
zero_interest = data[data["Interest Rate"] == 0].shape[0]
zero_interest

2437

In [10]:
#Loan type and status summary
loan_status = data['Loan Status'].value_counts()
loan_type= data['Loan Type'].value_counts()

5. #### Data cleaning

In [11]:
# Convert date time fields from  object data type to datetime data type

date_fields = ['End of Period', 'First Repayment Date', 'Last Repayment Date', 'Agreement Signing Date', 'Board Approval Date', 'Effective Date (Most Recent)', 'Closed Date (Most Recent)', 'Last Disbursement Date']
for field in date_fields:
    data[field] = pd.to_datetime(data[field])

In [12]:
data.columns = data.columns.str.replace("'s", "")
data.head(2)

,End of Period,Loan Number,Region,Country Code,Country,Borrower,Guarantor Country Code,Guarantor,Loan Type,Loan Status,...,Repaid 3rd Party,Due 3rd Party,Loans Held,First Repayment Date,Last Repayment Date,Agreement Signing Date,Board Approval Date,Effective Date (Most Recent),Closed Date (Most Recent),Last Disbursement Date
0,2020-11-30,IBRD00010,EUROPE AND CENTRAL ASIA,FR,France,CREDIT NATIONAL,FR,France,NPL,Fully Repaid,...,249962000.0,0.0,0.0,1952-11-01,1977-05-01,1947-05-09,1947-05-09,1947-06-09,1947-12-31,NaT
1,2020-11-30,IBRD00020,EUROPE AND CENTRAL ASIA,NL,Netherlands,NaN,NaN,NaN,NPL,Fully Repaid,...,87672000.0,0.0,0.0,1952-04-01,1972-10-01,1947-08-07,1947-08-07,1947-09-11,1948-03-31,NaT


In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8986 entries, 0 to 8985
Data columns (total 33 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   End of Period                 8986 non-null   datetime64[ns]
 1   Loan Number                   8986 non-null   object        
 2   Region                        8986 non-null   object        
 3   Country Code                  8984 non-null   object        
 4   Country                       8986 non-null   object        
 5   Borrower                      8929 non-null   object        
 6   Guarantor Country Code        8704 non-null   object        
 7   Guarantor                     8706 non-null   object        
 8   Loan Type                     8986 non-null   object        
 9   Loan Status                   8986 non-null   object        
 10  Interest Rate                 8865 non-null   float64       
 11  Currency of Commitment        

In [14]:
def  connect_database():
    
        engine_stmt = 'mysql+pymysql://root:@localhost/test'
        engine = sqlalchemy.create_engine(engine_stmt)
        conn = mysql.connector.connect(user='root', password='', host='127.0.0.1', database='test')
        #cursor = conn.cursor()
        return conn

In [15]:
def  country_data():
        engine = connect_database()
        country_df = data[['Country', 'Region', 'Country Code']].drop_duplicates(subset="Country Code")
        historical_countrydata = pd.read_sql("SELECT country FROM experian_country",engine)
        new_data = country_df[~country_df['Country'].isin(historical_countrydata['country'].tolist())].fillna("---")
        return new_data

In [16]:
def  country_service():
        country_dataframe = country_data()
        connection = connect_database()
        cursor = connection.cursor()
        for i,row in country_dataframe.iterrows():
            sql = "INSERT INTO experian_country (`country`, `region`, `country_code`)" " VALUES (%s,%s,%s )"
            values = (row[0], row[1], row[2])
            cursor.execute(sql, tuple(values)) 
            connection.commit()
            #cursor.close()
            #connection.close()

In [17]:
#country_service()

In [18]:
def  loan_data():
        engine = connect_database()
        loan_df = data[['End of Period', 'Loan Number', 'Country Code', 'Guarantor Country Code','Loan Type','Loan Status','Interest Rate','Currency of Commitment']].drop_duplicates(subset="Loan Number")
        historical_loandata = pd.read_sql("SELECT loan_number FROM experian_loan",engine)
        new_loan_data = loan_df[~loan_df['Loan Number'].isin(historical_loandata['loan_number'].tolist())].fillna("---")
        return new_loan_data

In [19]:
def  loan_service():
        loan_df = loan_data()
        connection = connect_database()
        cursor1 = connection.cursor()
        for i,row in loan_df.iterrows():
            sql = "INSERT INTO experian_loan(`end_period`,`loan_number` ,`country_code` ,`guarantor_country_code` ,`loan_type`,`loan_status`,`interest_rate` ,`currency_of_commitment`,`source_file_name`)" " VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            values = (row[0], row[1], row[2],row[3],row[4], row[5], row[6], row[7], full_file_name)
            cursor1.execute(sql, tuple(values)) 
            connection.commit()
            #cursor1.close()
            #connection.close()

In [20]:
#loan_service()

In [21]:
loan_data()

,End of Period,Loan Number,Country Code,Guarantor Country Code,Loan Type,Loan Status,Interest Rate,Currency of Commitment


In [22]:
def  project_data():
        engine = connect_database()
        project_df = data[['Loan Number','Project ID','Project Name','Original Principal Amount','Cancelled Amount','Undisbursed Amount','Disbursed Amount','Repaid to IBRD','Due to IBRD','Exchange Adjustment',           
                           'Borrower Obligation','Sold 3rd Party','Repaid 3rd Party','Due 3rd Party','Loans Held','First Repayment Date','Last Repayment Date','Agreement Signing Date','Board Approval Date',           
                           'Effective Date (Most Recent)','Closed Date (Most Recent)','Last Disbursement Date']]
        historical_projectdata = pd.read_sql("SELECT loan_id FROM experian_project",engine)
        new_project_data = project_df[~project_df['Loan Number'].isin(historical_projectdata['loan_id'].tolist())].fillna("---")
        return new_project_data

In [23]:
project_data()

,Loan Number,Project ID,Project Name,Original Principal Amount,Cancelled Amount,Undisbursed Amount,Disbursed Amount,Repaid to IBRD,Due to IBRD,Exchange Adjustment,...,Repaid 3rd Party,Due 3rd Party,Loans Held,First Repayment Date,Last Repayment Date,Agreement Signing Date,Board Approval Date,Effective Date (Most Recent),Closed Date (Most Recent),Last Disbursement Date
0,IBRD00010,P037383,RECONSTRUCTION,2.500000e+08,0.00,0.0,2.500000e+08,3.800000e+04,0.000000e+00,0.000000e+00,...,2.499620e+08,0.0,0.000000e+00,1952-11-01 00:00:00,1977-05-01 00:00:00,1947-05-09 00:00:00,1947-05-09,1947-06-09 00:00:00,1947-12-31 00:00:00,---
1,IBRD00020,P037452,RECONSTRUCTION,1.910442e+08,0.00,0.0,1.910442e+08,1.033722e+08,0.000000e+00,0.000000e+00,...,8.767200e+07,0.0,0.000000e+00,1952-04-01 00:00:00,1972-10-01 00:00:00,1947-08-07 00:00:00,1947-08-07,1947-09-11 00:00:00,1948-03-31 00:00:00,---
2,IBRD00021,P037452,RECONSTRUCTION,3.955788e+06,0.00,0.0,3.955788e+06,0.000000e+00,0.000000e+00,0.000000e+00,...,3.955788e+06,0.0,0.000000e+00,1953-04-01 00:00:00,1954-04-01 00:00:00,1948-05-25 00:00:00,1947-08-07,1948-06-01 00:00:00,1948-06-30 00:00:00,---
3,IBRD00030,P037362,RECONSTRUCTION,4.000000e+07,0.00,0.0,4.000000e+07,1.777100e+07,0.000000e+00,0.000000e+00,...,2.222900e+07,0.0,0.000000e+00,1953-02-01 00:00:00,1972-08-01 00:00:00,1947-08-22 00:00:00,1947-08-22,1947-10-17 00:00:00,1949-03-31 00:00:00,---
4,IBRD00040,P037451,RECONSTRUCTION,1.200000e+07,238016.98,0.0,1.176198e+07,1.619983e+06,-4.660000e-10,4.660000e-10,...,1.014200e+07,0.0,-4.660000e-10,1949-07-15 00:00:00,1972-07-15 00:00:00,1947-08-28 00:00:00,1947-08-28,1947-10-24 00:00:00,1949-03-31 00:00:00,---
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8981,IBRDS0160,P004894,SN METAL TECHNICAL ASST,2.000000e+07,20000000.00,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.0,0.000000e+00,1984-05-15 00:00:00,1989-11-15 00:00:00,---,1979-11-13,1979-11-13 00:00:00,1983-06-30 00:00:00,---
8982,IBRDS0170,P004895,WATER SUP & ENG.,5.000000e+06,1388308.92,0.0,3.611691e+06,3.611691e+06,0.000000e+00,0.000000e+00,...,0.000000e+00,0.0,0.000000e+00,1983-11-15 00:00:00,1989-11-15 00:00:00,1980-02-14 00:00:00,1980-01-15,1980-09-17 00:00:00,1984-12-31 00:00:00,1985-11-13 00:00:00
8983,IBRDS0180,P005392,PETROLEUM EXPLORATION I,5.000000e+07,103917.29,0.0,4.989608e+07,4.989608e+07,7.450580e-09,-7.450580e-09,...,0.000000e+00,0.0,7.450580e-09,1984-11-01 00:00:00,1997-05-01 00:00:00,1980-05-19 00:00:00,1980-04-15,1980-10-24 00:00:00,1984-06-30 00:00:00,---
8984,IBRDS0190,P007969,SIDERPERU TA,5.000000e+06,3643143.25,0.0,1.356857e+06,1.356857e+06,2.328310e-10,-2.328310e-10,...,0.000000e+00,0.0,2.328310e-10,1984-12-15 00:00:00,1990-12-15 00:00:00,1980-12-15 00:00:00,1980-07-15,1981-04-16 00:00:00,1986-06-30 00:00:00,1983-10-11 00:00:00


In [24]:
def  project_service():
        project_df = project_data()
        connection = connect_database()
        cursor = connection.cursor()
        for i,row in project_df.iterrows():
            sql = "INSERT INTO experian_project(`project_id`,`project_name`,`original_principal_amount`,`cancelled_amount`,`undisbursed_amount`,`disbursed_amount`,`repaid_to_IBRD`,`due_to_IBRD`,`exchange_adjustment`,`borrowers_obligation`,`sold_3rd_party`,`repaid_3rd_party`,`due_3rd_party`,`loans_held`,`first_repayment_date`,`last_repayment_date`,`agreement_signing_date`,`board_approval_date`,`effective_date`,`closed_date`, `last_disbursement_date`, `loan_id`)" " VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            values = (row[1], row[2],row[3],row[4], row[5], row[6], row[7], row[8], row[9], row[10],row[11],row[12], row[13], row[14], row[15],row[16],row[17], row[18], row[19], row[20],row[21],row[0])
            cursor.execute(sql, tuple(values)) 
            connection.commit()

In [25]:
project_service()